In [ ]:
%pip install path
%pip install opencv-python
%pip install --upgrade pyqt5 lxml

## Image labelling

In [2]:
!labelme

In [1]:
import os

labelimg_path = os.path.abspath(os.path.join("Tensorflow", "labelimg"))

!cd {labelimg_path} && pyrcc5 -o libs/resources.py resources.qrc
!cd {labelimg_path} && python labelImg.py

## Data augmentation

In [ ]:
from main import files, paths
import os

images_dir = os.path.join(paths["IMAGE_PATH"], "train")
out_dir = os.path.join(paths["IMAGE_PATH"], "augmented")

cmd = f"""python {files["DATA_AUGM_SCRIPT"]} 
        --images_dir="{images_dir}"
        --out_dir="{out_dir}"
    """

cmd = " ".join([s.strip() for s in str(cmd).split()])


print(cmd)

# !{cmd}

## Generate JSON files

In [ ]:
import os
from main import paths

images_path = paths["IMAGE_PATH"]
annotation_path = paths["ANNOTATION_PATH"]

train_img_dir = os.path.join(images_path, "train")
val_img_dir = os.path.join(images_path, "val")

train_json_file = os.path.join(train_img_dir, "train.json")
val_json_file = os.path.join(val_img_dir, "val.json")

if(os.path.exists(train_json_file)):
    os.remove(train_json_file)

if(os.path.exists(val_json_file)):
    os.remove(val_json_file)

train_cmd = f"""python src/labelme2coco.py "{train_img_dir}"
    --output="{train_json_file}"
    """

train_cmd = " ".join([s.strip() for s in str(train_cmd).split()])

print(train_cmd)
!{train_cmd}

val_cmd = f"""python src/labelme2coco.py "{val_img_dir}"
    --output="{val_json_file}"
    """

val_cmd = " ".join([s.strip() for s in str(val_cmd).split()])

print(val_cmd)
!{val_cmd}

## Generate TFRecord files

In [ ]:
import os
from main import paths

images_path = paths["IMAGE_PATH"]
annotation_path = paths["ANNOTATION_PATH"]

train_img_dir = os.path.join(images_path, "train")
val_img_dir = os.path.join(images_path, "val")

train_json_file = os.path.join(train_img_dir, "train.json")
val_json_file = os.path.join(val_img_dir, "val.json")

tf_record_Cmd = f"""python src/create_coco_tf_record.py 
                --logtostderr
                --train_image_dir="{os.path.abspath(os.path.join(images_path, "train"))}"
                --val_image_dir="{os.path.abspath(os.path.join(images_path, "val"))}"
                --train_annotations_file="{train_json_file}"
                --val_annotations_file="{val_json_file}"
                --include_masks={True}
                --output_dir="{annotation_path}"
                """
tf_record_Cmd = " ".join([s.strip() for s in str(tf_record_Cmd).split()])

print(tf_record_Cmd)
!{tf_record_Cmd}

In [22]:
# For XML Bounding Boxes
from main import files, paths
import os 

train_cmd = f"""python {files["TF_RECORD_SCRIPT"]}
        -x="{os.path.abspath(os.path.join(paths['IMAGE_PATH'], 'augmented'))}"
        -l="{os.path.abspath(files['LABEL_MAP'])}"
        -o="{os.path.abspath(os.path.join(paths['ANNOTATION_PATH'], 'train.record'))}"
        """

train_cmd = " ".join([s.strip() for s in str(train_cmd).split()])

!{train_cmd}

val_cmd = f"""python {files["TF_RECORD_SCRIPT"]}
        -x="{os.path.abspath(os.path.join(paths['IMAGE_PATH'], 'val'))}"
        -l="{os.path.abspath(files['LABEL_MAP'])}"
        -o="{os.path.abspath(os.path.join(paths['ANNOTATION_PATH'], 'val.record'))}"
        """

val_cmd = " ".join([s.strip() for s in str(val_cmd).split()])

!{val_cmd}

Successfully created the TFRecord file: c:\Users\jtorn\Desktop\Dev\Redes neuronales e AI\ray_c\Tensorflow\workspace\annotations\train.record


c:\Users\jtorn\miniconda3\envs\ray_c\lib\site-packages\google\auth\crypt\_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography (40.0) will be the last to support Python 3.6.
  import cryptography.exceptions


Successfully created the TFRecord file: c:\Users\jtorn\Desktop\Dev\Redes neuronales e AI\ray_c\Tensorflow\workspace\annotations\val.record


c:\Users\jtorn\miniconda3\envs\ray_c\lib\site-packages\google\auth\crypt\_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography (40.0) will be the last to support Python 3.6.
  import cryptography.exceptions


## Train the model

In [ ]:
import os
from main import paths, files

optimized_ckpt = None
# optimized_ckpt = os.path.join(paths['CHECKPOINT_PATH'], "export", "checkpoint")
optimized_config = None
# optimized_config = os.path.join(
    # paths['CHECKPOINT_PATH'], "export", "pipeline.config")

TRAINING_SCRIPT = os.path.join(
    paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

steps = 4000

command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(
    TRAINING_SCRIPT, optimized_ckpt or paths['CHECKPOINT_PATH'], optimized_config or files['PIPELINE_CONFIG'], steps)

print(command)

#!{command}

## Obtain metrics (loss, precision)

In [ ]:
import os
from main import paths, files

TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'], files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

print(command)

# !{command}

## Analyze metrics with Tensorboard

In [26]:
import os
from main import paths

host_url = "http://localhost:6006/"

metrics_type = "eval" # train - eval

log_dir = os.path.join(paths["CHECKPOINT_PATH"], metrics_type)

command = f"tensorboard --logdir={log_dir}"

print(command)
print(f"Going to be ready on => {host_url}")
# !{command}

tensorboard --logdir=Tensorflow\workspace\models\model\eval
Going to be ready on => http://localhost:6006/


## Saving the model

In [ ]:
import os
from main import paths, files

FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')
# command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

print(command)

!{command}

## Optimizing:

In [ ]:
import tensorflow as tf
from main import paths
import numpy as np
import cv2

optimizer_images = os.path.join(paths["IMAGE_PATH"], "to_optimize_model")

# TODO: Make this work XD
def representative_dataset_gen():
  for _ in range(1, 5):
    # Get sample input data as a numpy array in a method of your choosing.
    image_path = os.path.join(optimizer_images, f'optimize ({_}).jpg')

    img = cv2.imread(image_path)

    if(img is None):
        continue

    print(f"IMAGE => {image_path}", img.size)   

    img = cv2.imread(image_path)

    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    yield [input_tensor]

converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(paths['OUTPUT_PATH'], "saved_model"))
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset_gen

tflite_quant_model = converter.convert()


## Use the model

In [ ]:
sd_ckpt_name = "urpm-inpainting.safetensors [b897946c3d]"

image_name = "14.png"

text_prompt = ""
negative_prompt = ""
steps = 28
fill_mode = "original"
output_dir = "out"

target_image_path = "C:\\Users\\jtorn\\Desktop\\Dev\\Redes neuronales e AI\\ray_c\\Tensorflow\\workspace\\images\\test\\" + image_name

command = f"""python src/inpaint_regression.py 
            --sd_ckpt_name="{sd_ckpt_name}"
            --image_path="{target_image_path}"
            --text_prompt="{text_prompt}"
            --negative_prompt="{negative_prompt}"
            --output_dir="{output_dir}"
            """

command = " ".join([s.strip() for s in str(command).split()])

print(command)

# !{command}
